# CS202: Compiler Construction

## In-class Exercises, Week of 02/27/2023

----

# RCO

## Question 1

How do we handle `if` statements in rco?

We don't have to do much at all

if is not an operator, it's arguments don't need to be atomic
- call rco_exp on the condition
- call rco_exp on the then and sles statements

----

# Explicate-Control

## Question 2

Describe the idea behind the explicate-control pass.

The pass is designed to compile if statements into a control flow graph(goto/jumps; no structural if statements)

The output is in an intermediate language called Cif.
- Similar to Python in most respects (expressions and statements)
- Different
    - If statements must have gotos in then and else
    - Program is represented by a dict mapping labels to lists of statements (a control flow graph)
    
Four pieces to considerwhen compiling an if statement:
- the condition
- the then branch (statements)
- the else branch (statements)
- the "rest of the stuff to do" after the if statement is done
    - this is called the *continuation*
    
We will process statements backwards, so we always have a compiled version of the continuation.

## Question 3

Compile this program into a $C_{if}$ program, describing the input and output of explicate-control.

```
if 5 == 6:
    print(1)
else:
    print(0)
```

```
label_2:
    return 0
label_3:
    print(1)
    goto label_2
label_4:
    print(0)
    goto label_2
start:
    tmp_1 = eq(5, 6)
    if tmp_1: goto label_3 else: goto label_4
```

## Question 4

Describe the structure of `explicate_control`.

Start by following the grammar
- `ec_atm` compiles an atomic expressin into a Cif atom
    - Constants => cif.Constant
    - Var => cif.Var
- `ec_expr` cmpiles an expression into a Cif expression
    - Prim(op, args) => cif.Prin(op, args)
    - else call ec_atm
- `ec_stmt` taes a stmt and a continuation, returns list of Cif statements
    - Assign(x, e) => [cif.Assign(x, ec_expr(e))] + continuation
    - Print(e) => [cif.Print(ec_expr(e))] + continuation
    - If(condition, then_stmts, else_stmts) => 
        - cond_label = create block for continuation
        - then_label = create block for ec_stmts(then_stmts, [cif.Goto(cond_label)])
        - else_label = create block for ec_stmts(else_stmts, [cif.Goto(cond_label)])
        - return [cif.If(ec_expr(condition), cif.Goto(the_label), cif.Goto(else_label))]
- `ec_stmts` takes a list of statements and a continuation, returns a list of Cif statements
    - process the list of statements in reverse
    - update "current continuation" by calling sc_stmt n each stmt and setting the continuation to whatever comes back
- the main body of the pass
    - start with the coontinuation: [cif.Return(0)]
    - call ec_stmts on the statements of the program
    - set basic_blocks['start'] to the result
- pass should have a "global" basic_blocks: Dict[str, List[cif.Stmt]]
- pass should have a `create_block` function that adds a new block to basic_blocks with a unique name (using gensym)
    

# Select Instructions

## Question 5

Describe the new cases in `select_instructions`.

- Mostly compy paste
- Comparison case is really interesting

New cases:
- `Assign(x, Prim(op, [a1, a2]))` and `op` is a comparison primitive:
    - compile to three instructions:
        - cmpq a2, a1 (compares a1 to a2, sets EFLAGS register)
        - set[cc] %al (sets byte register %al to the value of EFLAGS)
            - cc is the condition code for the desired comparison operation
            - eq:e, gt:g, gte: ge, lt: l, lte: le
        - movxbq %al, #x (moves byte register into a 64-bit location)
- `If(condition, Goto(label1), Goto(label2))`
    - cmpq condition, `$1`
    - je label1
    - jmp label2
- `Assign(x, Prim('not', [a1]))`
    - movq a1, #x
    - xorq $1, #x
- `Return(a1)`
    - movq a1, %rax
    - jmp conclusion
- `Goto(label)` => jmp label

----

# Register Allocation

## Question 6

Describe the liveness analysis for a control-flow graph.

- Challenge: now I have many blocks, and jumps between them; a variable might be read in a block I'm jumping to, and so it better remain live "across" the jump
- This means I can't consider each block in isolation, I need to look at all blocks at once 
- Treat each block as a sort of temporary variable

## Question 7

Here is an x86 assembly program. Compute its live-after sets.

```
start:
  movq $42, #x
  cmpq $6, $5
  je label_2
  jmp label_3
label_2:
  addq $3, #x
  jmp conclusion
label_3:
  addq $4, #x
  jmp conclusion
```

When I find a jump instruction, compute the live after sets for the target, and the live before set for the target, and treat the live before set as a variable read by the jump

The conclusion block always have the empty live before set.

```
start:         {}
  movq $42, #x {x}
  cmpq $6, $5  {x}
  je label_2   {x}
  jmp label_3  {}
label_2:         {x}
  addq $3, #x    {}
  jmp conclusion {}
label_3:         {x}
  addq $4, #x    {}
  jmp conclusion {}
```

- Global dict that remembers live before sets for each block
- Global dict that remembers list of live after sets for each block

## Question 8

Describe the change to building the interference graph.

No change

## Question 9

Describe the change to graph coloring and assigning homes.

No change

----

# Patch Instructions

## Question 10

Describe the changes to patch_instructions.

- basically same as a3
- cmpq instr: cant have immediate as second instruction

----

# Prelude & Conclusion


## Question 11

Describe the changes to prelude_and_conclusion.

- almost same as a3
- prelude code goes in `main`
- conclusion code goes in `conclusion`